In [ ]:
!ls ../../..

In [ ]:
!pip install tensorflow-object-detection-api

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
!git clone https://github.com/tensorflow/models

In [ ]:
%cd /kaggle/working/
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.tar.gz
!tar -xvf ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.tar.gz
!rm ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.tar.gz

In [ ]:
%cd /kaggle/working/models/research
!protoc object_detection/protos/*.proto --python_out=.
# Install TensorFlow Object Detection API.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .
!pip install tensorflow==2.13.0

In [ ]:
!python /kaggle/working/models/research/object_detection/builders/model_builder_tf2_test.py

In [ ]:
!pip install wget

In [ ]:
!wget https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.config
!mv ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.config /kaggle/working/ssd_resnet101.config

In [ ]:
num_classes = 12
batch_size = 4
num_steps = 20000
num_eval_steps = 1000

train_record_path = '/kaggle/input/tfrecord/train.tfrecord'
test_record_path = '/kaggle/input/tfrecord/test.tfrecord'
model_dir = '/kaggle/working/training/'
labelmap_path = '/kaggle/input/labelmap/labelmap.pbtxt'

pipeline_config_path = '/kaggle/working/ssd_resnet101.config'
fine_tune_checkpoint = '/kaggle/working/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0'

In [ ]:
import re

with open(pipeline_config_path) as f:
    config = f.read()

with open(pipeline_config_path, 'w') as f:

    # Set labelmap path
    config = re.sub('label_map_path: ".*?"',
                  'label_map_path: "{}"'.format(labelmap_path), config)

    # Set fine_tune_checkpoint path
    config = re.sub('fine_tune_checkpoint: ".*?"',
                  'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), config)

    # Set fine_tune_checkpoint typr
    config = re.sub('fine_tune_checkpoint_type: ".*?"',
                  'fine_tune_checkpoint_type: "{}"'.format('detection'), config)

    # Set train tf-record file path
    config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")',
                  'input_path: "{}"'.format(train_record_path), config)

    # Set test tf-record file path
    config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")',
                  'input_path: "{}"'.format(test_record_path), config)

    # Set number of classes.
    config = re.sub('num_classes: [0-9]+',
                  'num_classes: {}'.format(num_classes), config)

    # Set batch size
    config = re.sub('batch_size: [0-9]+',
                  'batch_size: {}'.format(batch_size), config)

    # Set training steps
    config = re.sub('num_steps: [0-9]+',
                  'num_steps: {}'.format(num_steps), config)
    
    # Set learning rate
   # config = re.sub('learning_rate_base: [.0-9]+',
    #              'learning_rate_base: {}'.format(learning_rate), config)

    f.write(config)

In [ ]:
import re

# Hapus seluruh blok konfigurasi optimizer momentum
config = re.sub(r'optimizer\s*{[^}]*}', '', config)

    # Pengaturan optimizer Adam yang akan digunakan
rms_prop_optimizer_config = """
    optimizer {
      rms_prop_optimizer: {
        learning_rate: {
          cosine_decay_learning_rate {
            learning_rate: 0.001  # Sesuaikan dengan kebutuhan Anda
          }
        }
      }
      use_moving_average: false
    }
    """

    # Gabungkan pengaturan optimizer Adam dengan konfigurasi yang ada
config += rms_prop_optimizer_config

print(config)

In [ ]:
!python /kaggle/working/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_config_path} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1 \
    #--checkpoint_every_n=100 \
    --num_eval_steps={num_eval_steps}

# VALIDATION

In [ ]:
!python /kaggle/working/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_config_path} \
    --model_dir={model_dir} \
    --checkpoint_dir={model_dir}

In [ ]:
output_directory = '/kaggle/working/inference_graph'

!python /kaggle/working/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir={model_dir} \
    --output_directory={output_directory} \
    --pipeline_config_path={pipeline_config_path}

# **TFLITE**

In [ ]:
!python /kaggle/working/models/research/object_detection/export_tflite_graph_tf2.py \
    --pipeline_config_path={pipeline_config_path} \
    --trained_checkpoint_dir /kaggle/working/training \
    --output_directory /kaggle/working/inference_graph_tflite

In [ ]:
%cd /kaggle/working/inference_graph_tflite
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model('/kaggle/working/inference_graph_tflite/saved_model') # path to the SavedModel directory
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()
# Save the TensorFlow Lite model to a file
with open('model17.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
!zip -r /kaggle/working/inference_graph.zip /kaggle/working/inference_graph/
!zip -r /kaggle/working/inference_graph_tflite.zip /kaggle/working/inference_graph_tflite/
!zip -r /kaggle/working/training.zip /kaggle/working/training/

# **TENSORBOARD**

In [ ]:
%load_ext tensorboard
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ./ngrok-stable-linux-amd64.zip
!./ngrok authtoken 2WR6sz0LL5J47kwf2UYMNr1L1fG_7AGSSvhLnEBtEGNvyz8Bw #ganti authken e signup ng ngrok

In [ ]:
 # Launch and tunnel tensorboard
import os
import multiprocessing
pool = multiprocessing.Pool(processes = 10)
results_of_processes = [pool.apply_async(os.system, args=(cmd, ), callback = None )
                        for cmd in [
                        f"tensorboard --logdir /kaggle/working/training --host 0.0.0.0 --port 6009 &",
                        "./ngrok http 6009 &"
                        ]]

In [ ]:
import time
time.sleep(10) # wait for tensorboard host
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

# ****TESTING****

In [ ]:
import os
import subprocess
from collections import namedtuple
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
%cd /kaggle/working/

In [ ]:
!wget https://raw.githubusercontent.com/hugozanini/object-detection/master/inferenceutils.py

In [ ]:
import sys

sys.path.insert(0, '/kaggle/working/')

from inferenceutils import *

In [ ]:
output_directory = 'inference_graph/'

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(labelmap_path, use_display_name=True)
tf.keras.backend.clear_session()
model = tf.saved_model.load(f'/kaggle/working/{output_directory}/saved_model')

# **Testing Image**

In [ ]:
images = ["0156.jpg", "0240.jpg", "0422.jpg"]

for image_name in images:
    image_np = load_image_into_numpy_array('/kaggle/input/testing/' + image_name)
    output_dict = run_inference_for_single_image(model, image_np)
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks = output_dict.get('detection_masks_reframed', None),
        use_normalized_coordinates = True,
        min_score_thresh = .46,
        line_thickness = 6)
    display(Image.fromarray(image_np))

# **Testing Video**

In [ ]:
import os
import time
import tensorflow as tf
import cv2
import numpy as np

from object_detection.utils import label_map_util
from PIL import Image
#from google.colab.patches import cv2_imshow
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

from IPython.display import HTML
from base64 import b64encode

#Path to saved model

PATH_TO_SAVED_MODEL = "/kaggle/working/inference_graph/saved_model"

# Load label map and obtain class names and ids
#label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
category_index=label_map_util.create_category_index_from_labelmap("/kaggle/input/images2/images/labelmap.pbtxt",use_display_name=True)
num_detect = [0,0,0,0]

def visualise_on_image(image, bboxes, labels, scores, thresh):
    (h, w, d) = image.shape
    for bbox, label, score in zip(bboxes, labels, scores):
        if score > thresh:
            xmin, ymin = int(bbox[1]*w), int(bbox[0]*h)
            xmax, ymax = int(bbox[3]*w), int(bbox[2]*h)

            cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (0,255,0), 2)
            cv2.putText(image, f"{label}: {int(score*100)} %", (xmin, ymin), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 2)
            if label == "R02":
              num_detect[0] += 1
              cv2.putText(image, f"1: {num_detect[0]}", (50,70), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (182, 3, 252), 2)
            if label == "R03":
              num_detect[1] += 1
              cv2.putText(image, f"2: {num_detect[1]}", (60,70), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (182, 3, 252), 2)
            if label == "L00":
              num_detect[2] += 1
              cv2.putText(image, f"3: {num_detect[2]}", (70,70), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (182, 3, 252), 2)
            if label == "00":
              num_detect[3] += 1
              cv2.putText(image, f"4: {num_detect[3]}", (80,70), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (182, 3, 252), 2)
            #print(label)
            #print(type(label))
    return image

if __name__ == '__main__':

    # Load the model
    print("Loading saved model ...")
    detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)
    print("Model Loaded!")

    # Video Capture (video_file)
    video_capture = cv2.VideoCapture("/kaggle/input/videotesting/output_test.avi")
    start_time = time.time()

    frame_width = int(video_capture.get(3))
    frame_height = int(video_capture.get(4))
    #fps = int(video_capture.get(5))
    size = (frame_width, frame_height)

    #Initialize video writer
    result = cv2.VideoWriter('/kaggle/working/output_test.avi', cv2.VideoWriter_fourcc(*'MJPG'),15, size)

    while True:
      ret, frame = video_capture.read()
      if not ret:
          print('Unable to read video / Video ended')
          break

      frame = cv2.flip(frame, 1)
      image_np = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
      # The model expects a batch of images, so also add an axis with `tf.newaxis`.
      input_tensor = tf.convert_to_tensor(image_np)[tf.newaxis, ...]

      # Pass frame through detector
      detections = detect_fn(input_tensor)

      # Set detection parameters

      score_thresh = 0.4   # Minimum threshold for object detection
      max_detections = 100

      # All outputs are batches tensors.
      # Convert to numpy arrays, and take index [0] to remove the batch dimension.
      # We're only interested in the first num_detections.
      scores = detections['detection_scores'][0, :max_detections].numpy()
      bboxes = detections['detection_boxes'][0, :max_detections].numpy()
      labels = detections['detection_classes'][0, :max_detections].numpy().astype(np.int64)
      labels = [category_index[n]['name'] for n in labels]

      # Display detections
      visualise_on_image(frame, bboxes, labels, scores, score_thresh)

      end_time = time.time()
      fps = int(1/(end_time - start_time))
      start_time = end_time
      cv2.putText(frame, f"FPS: {fps}", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)
      #cv2_imshow(frame)
      #print ([category_index.get(value) for index,value in enumerate(labels[0]) if scores[0] > 0.4])
      #Write output video
      result.write(frame)

    for i in num_detect:
        print(i, end=" ")
    video_capture.release()